In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

### Okun's Law

- `g`: growth rate of output
- `u`: unemployment rate

In [2]:
okun = pd.read_sas('okun.sas7bdat')

In [3]:
print(okun.head())

     g    u
0  1.4  7.3
1  2.0  7.2
2  1.4  7.0
3  1.5  7.0
4  0.9  7.2


### Autocorrelation Coefficient

We import `acf` from `statsmodels.tsa.stattools`

In [4]:
from statsmodels.tsa.stattools import acf

In [5]:
g_acf = acf(okun['g'],fft=False)
print(g_acf)

[ 1.          0.49425676  0.4107073   0.1544205   0.20043788  0.09038538
  0.02447111 -0.03008434 -0.08231978  0.04410661 -0.02128483 -0.08683463
 -0.20404326 -0.14418367 -0.06948648 -0.06402047 -0.08179742 -0.08479459
 -0.07667928  0.00103167 -0.05466495 -0.10768211 -0.04507015  0.0237855
  0.12412861  0.11713235  0.13231832  0.04490049  0.13946239  0.19462665
  0.17964479  0.06940628  0.0813598   0.05801531  0.02024042 -0.03398073
 -0.14582792 -0.09337329 -0.06815819  0.05394989 -0.01896214]


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:655: FutureWarning: The default number of lags is changing from 40 tomin(int(10 * np.log10(nobs)), nobs - 1) after 0.12is released. Set the number of lags to an integer to  silence this warning.
  warnings.warn(


### Philips Curve of Australia

- `inf`: inflation rate
- `u`: unemployment rate

Please note that `inf` is a reserved word in Python, which means infinity. We should avoid this kind of potential confusion.

In [6]:
phillips_aus = pd.read_sas('phillips_aus.sas7bdat')

In [7]:
print(phillips_aus.head())

   inf    u
0  2.0  8.1
1  1.5  8.0
2  1.7  7.8
3  1.8  7.7
4  1.8  7.3


**Use `.shift()` to get the lagged variable

In [8]:
phillips_aus['du'] = phillips_aus['u'] - phillips_aus['u'].shift(1)

In [9]:
print(phillips_aus.head())

   inf    u   du
0  2.0  8.1  NaN
1  1.5  8.0 -0.1
2  1.7  7.8 -0.2
3  1.8  7.7 -0.1
4  1.8  7.3 -0.4


### Change the variable name `inf` to `y`

`inf` is a reserved word in Python. Its meaning is infinity. 

In [10]:
phillips_aus['y'] = phillips_aus['inf']

In [11]:
mdl_phillips_aus = smf.ols('y ~ du', data = phillips_aus).fit()
print(mdl_phillips_aus.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     5.295
Date:                Tue, 16 Feb 2021   Prob (F-statistic):             0.0238
Time:                        20:01:11   Log-Likelihood:                -83.958
No. Observations:                  90   AIC:                             171.9
Df Residuals:                      88   BIC:                             176.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.7776      0.066     11.813      0.0

In [12]:
phillips_aus['e']=mdl_phillips_aus.resid

In [13]:
e_acf = acf(phillips_aus['e'][1:],fft=False)
print(e_acf)

[ 1.          0.54865864  0.45573248  0.43321578  0.42049358  0.33903419
  0.27097344  0.1912208   0.250694    0.15340865  0.05000153 -0.01573694
 -0.01318805 -0.00856231 -0.00350055 -0.12810175 -0.12837491  0.00937824
 -0.01553458  0.01229718  0.0514645   0.0872629   0.10266355  0.01361149
  0.01581844  0.02512371 -0.03429133 -0.05338998 -0.05255196 -0.10977013
 -0.10840154 -0.1095806  -0.14186622 -0.21696352 -0.21653986 -0.2266333
 -0.24662648 -0.21327263 -0.1644506  -0.15469006 -0.12907371]


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:655: FutureWarning: The default number of lags is changing from 40 tomin(int(10 * np.log10(nobs)), nobs - 1) after 0.12is released. Set the number of lags to an integer to  silence this warning.
  warnings.warn(


In [14]:
phillips_aus['le'] = phillips_aus['e'].shift(1)

In [15]:
print(phillips_aus.head())

   inf    u   du    y         e        le
0  2.0  8.1  NaN  2.0       NaN       NaN
1  1.5  8.0 -0.1  1.5  0.669592       NaN
2  1.7  7.8 -0.2  1.7  0.816806  0.669592
3  1.8  7.7 -0.1  1.8  0.969592  0.816806
4  1.8  7.3 -0.4  1.8  0.811233  0.969592


In [16]:
mdl_phillips_aus2 = smf.ols('y~du+le', data=phillips_aus).fit()

print(mdl_phillips_aus2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.349
Model:                            OLS   Adj. R-squared:                  0.334
Method:                 Least Squares   F-statistic:                     23.04
Date:                Tue, 16 Feb 2021   Prob (F-statistic):           9.73e-09
Time:                        20:01:11   Log-Likelihood:                -66.400
No. Observations:                  89   AIC:                             138.8
Df Residuals:                      86   BIC:                             146.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.7682      0.055     13.911      0.0

In [17]:
mdl_phillips_aus3 = smf.ols('e~du+le', data=phillips_aus).fit()

print(mdl_phillips_aus3.summary())

                            OLS Regression Results                            
Dep. Variable:                      e   R-squared:                       0.310
Model:                            OLS   Adj. R-squared:                  0.294
Method:                 Least Squares   F-statistic:                     19.34
Date:                Tue, 16 Feb 2021   Prob (F-statistic):           1.16e-07
Time:                        20:01:11   Log-Likelihood:                -66.400
No. Observations:                  89   AIC:                             138.8
Df Residuals:                      86   BIC:                             146.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0094      0.055     -0.170      0.8

### HAC SE

For HAC standard error, the closest to the textbook version we can find is to use `cov_type='hac', cov_kwds={'maxlags':3}` as parameters inside `.fit()`.

In [18]:
mdl_phillips_aus4 = smf.ols('y~du', data=phillips_aus).fit(cov_type='hac', cov_kwds={'maxlags':3})

print(mdl_phillips_aus4.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     2.914
Date:                Tue, 16 Feb 2021   Prob (F-statistic):             0.0913
Time:                        20:01:11   Log-Likelihood:                -83.958
No. Observations:                  90   AIC:                             171.9
Df Residuals:                      88   BIC:                             176.9
Df Model:                           1                                         
Covariance Type:                  hac                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.7776      0.102      7.635      0.0